In [7]:
import torch
import torchvision.models as models
from  model import generate_model
import resnet

In [47]:
# model = generate_model()

In [141]:
model = resnet.resnet10(sample_input_D=14,sample_input_H=28,sample_input_W=28,num_seg_classes=10)
checkpoint = torch.load(r'C:\Users\hello\Downloads\MedicalNet_pytorch_files2\pretrain\resnet_10_23dataset.pth')
# model.load_state_dict(checkpoint['state_dict'])
# print(checkpoint)

# for param in model.parameters():
#     param.requires_grad = False
# model.load_state_dict(r'C:\Users\hello\Downloads\MedicalNet_pytorch_files2\pretrain\resnet_10.pth')
#freeze
#nn.linear  默认解冻
#unfreeze

In [142]:
model

ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, 

In [50]:
model.conv_seg = torch.nn.Linear(512,10)


In [110]:
class MedicalNet(nn.Module):

  def __init__(self, path_to_weights,device):
    super(MedicalNet, self).__init__()
    self.model = resnet.resnet10(sample_input_D=64, sample_input_H=96, sample_input_W=96, num_seg_classes=2)
    self.model.conv_seg = nn.Sequential(
                                        nn.AdaptiveMaxPool3d(output_size=(1, 1, 1)),
                                        nn.Flatten(start_dim=1),
                                        nn.Dropout(0.1),
                                        nn.Linear(512, 2)
                                        )#修改分割层为线性层
    net_dict = self.model.state_dict()
    pretrained_weights = torch.load(path_to_weights, map_location=torch.device(device))
    pretrain_dict = {
        k.replace("module.", ""): v for k, v in pretrained_weights['state_dict'].items() if k.replace("module.", "") in net_dict.keys()
      }
    net_dict.update(pretrain_dict)
    self.model.load_state_dict(net_dict)
    print( self.model.load_state_dict(net_dict))
    self.fc = nn.Linear(512, 2)

  def forward(self, x):
    features = self.model(x)
#     return self.fc(features)
    return features

In [139]:
for param_name, param in model.named_parameters():
#     print(param_name.split('.')[0] )
    
    if param_name.split('.')[0]=="conv_seg":
        print(param_name)
        param.requires_grad = True
    else:
        param.requires_grad = False

conv_seg.0.weight
conv_seg.0.bias
conv_seg.1.weight
conv_seg.1.bias
conv_seg.3.weight
conv_seg.4.weight
conv_seg.4.bias
conv_seg.6.weight


In [112]:
import torch.nn as nn 
import torch
from torch.autograd import Variable


In [113]:
device=torch.device('cuda')
model = MedicalNet(path_to_weights=r"C:\Users\hello\Downloads\MedicalNet_pytorch_files2\pretrain\resnet_10.pth",device=device)
img = Variable(torch.rand(16, 1, 96, 96, 64))
output=model(img)
output.shape

  if param_name.split('.')[0]=="conv_seg":
    print(param_name)
    param.requires_grad = True
  else:
    param.requires_grad = False
#     print(param_name)
    
#     print(param_name.startswith("conv_seg"))
    

<All keys matched successfully>


In [140]:
model

ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, 

In [115]:
output.shape

torch.Size([16, 2])

In [132]:
# model
path_to_weights = r"C:\Users\hello\Downloads\MedicalNet_pytorch_files2\pretrain\resnet_10.pth"
model = resnet.resnet10(sample_input_D=64, sample_input_H=96, sample_input_W=96, num_seg_classes=2)
net_dict = model.state_dict()#网络结构参数
pretrained_weights = torch.load(path_to_weights, map_location=torch.device(device))#加载预训练模型权重文件
pretrain_dict = {
        k.replace("module.", ""): v for k, v in pretrained_weights['state_dict'].items() if
        k.replace("module.", "") in net_dict.keys()
    }
print(pretrained_weights)
# for k, v in pretrained_weights['state_dict'].items():
#     print(k,v)#module.conv1.weight 
    
# pretrain_dict = {
#         k.replace("module.", ""): v for k, v in pretrained_weights['state_dict'].items() if
#         k.replace("module.", "") in net_dict.keys()
#     }
# net_dict.update(pretrain_dict)
# model.load_state_dict(pretrained_weights)

{'state_dict': OrderedDict([('module.conv1.weight', tensor([[[[[ 3.5011e-01,  3.7391e-01,  4.7203e-01,  ...,  4.3368e-01,
             9.4722e-01,  5.3238e-01],
           [ 6.0360e-01,  7.1614e-01,  6.0511e-01,  ...,  1.3093e-01,
             5.0663e-01,  7.4470e-01],
           [ 8.2122e-01,  7.5440e-01,  6.7305e-01,  ...,  7.5137e-01,
             8.5558e-01,  7.2442e-01],
           ...,
           [ 1.2660e+00,  1.0226e+00,  7.2709e-01,  ...,  5.2497e-01,
             8.3902e-01,  6.9709e-01],
           [ 1.4075e+00,  1.1948e+00,  9.8492e-01,  ...,  5.7407e-01,
             8.6237e-01,  8.2963e-01],
           [ 1.1119e+00,  8.2790e-01,  8.7238e-01,  ...,  5.3499e-01,
             9.1795e-01,  7.7600e-01]],

          [[-3.4746e-02, -8.5801e-02, -3.9745e-02,  ..., -2.3789e-02,
             5.1352e-01,  2.5380e-01],
           [ 1.1587e-02,  4.0656e-01,  2.8453e-01,  ..., -2.5139e-01,
             1.7467e-01,  3.7206e-01],
           [ 3.3199e-01,  2.3296e-01,  2.3651e-01,  ...,  

In [119]:
path_to_weights = r"C:\Users\hello\Downloads\MedicalNet_pytorch_files2\pretrain\resnet_10.pth"
model = resnet.resnet10(sample_input_D=64, sample_input_H=96, sample_input_W=96, num_seg_classes=2)
model.conv_seg = nn.Sequential(
        nn.AdaptiveMaxPool3d(output_size=(1, 1, 1)),
        nn.Flatten(start_dim=1),
#         nn.Dropout(0.1),
        nn.Linear(512,2)

    )  # 修改分割层为线性层

In [122]:
net_dict = model.state_dict()
pretrained_weights = torch.load(path_to_weights, map_location=torch.device(device))
pretrain_dict = {
        k.replace("module.", ""): v for k, v in pretrained_weights['state_dict'].items() if
        k.replace("module.", "") in net_dict.keys()
    }
net_dict.update(pretrain_dict)
model.load_state_dict(net_dict)

<All keys matched successfully>

In [124]:
model.load_state_dict(net_dict)

<All keys matched successfully>